# Step 1: Data preprocess

## User Defined Functions

In [4]:
def intersect(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

## **Notebook Starts from here**

In [ ]:
#Necessary imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import numpy as np

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
#Loading meta data for gene expression
sample_info=pd.read_csv('Data/GSE96058-GPL11154.txt',delimiter='\t',skiprows=60)

In [18]:
#loading gene expresssion

count_data=pd.read_csv('Data/GSE96058_gene_expression_transformed.csv')

In [19]:
print(count_data.shape)
count_data.head()

(30865, 3410)


,Unnamed: 0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,...,F2390repl,F2398repl,F2406repl,F2407repl,F2408repl,F2424repl,F2459repl,F2482repl,F2503repl,F2539repl,F2621repl,F2625repl,F2632repl,F2656repl,F2679repl,F2691repl,F2703repl,F2755repl,F2781repl,F2790repl,F2840repl,F2841repl,F2860repl,F2871repl,F2872repl,F2877repl,F2911repl,F2912repl,F2921repl,F2958repl,F2974repl,F3006repl,F3028repl,F3057repl,F3058repl,F3085repl,F3127repl,F3135repl,F3250repl,F3265repl
0,5_8S_rRNA,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
1,5S_rRNA,4.911099,-3.321928,-3.321928,3.656393,4.190104,2.556304,2.590351,5.691788,-3.321928,3.223401,-3.321928,5.265988,4.179053,3.936864,4.322433,3.903810,3.070013,3.055105,6.280728,-3.321928,-3.321928,-3.321928,3.090068,-3.321928,4.208509,-3.321928,-3.321928,3.215818,2.083955,5.462131,3.962688,2.411087,-3.321928,2.839232,3.663231,5.147938,-3.321928,-3.321928,1.912470,...,-3.321928,-3.321928,-3.321928,5.294816,5.379012,4.957720,-3.321928,7.154195,-3.321928,5.130144,-3.321928,3.717759,-3.321928,5.646126,5.919540,4.272025,3.947339,3.372381,-3.321928,-3.321928,-3.321928,-3.321928,3.468127,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,1.614206,4.627216,4.084251,2.287523,3.205371,6.965451,-3.321928,4.531984,-3.321928,5.097337,1.895737,2.954997
2,6M1-18,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
3,7M1-2,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
4,7SK,-0.539253,-0.576620,-1.651323,0.126633,0.783715,-1.759556,-1.033968,-0.129513,0.494308,0.749841,1.541830,0.049312,1.260500,-0.423132,-0.478065,2.304028,0.343800,-0.268999,0.122726,4.734924,-0.639770,0.224191,-0.269517,1.563199,1.033623,0.905147,0.262900,0.877646,1

In [20]:
print(sample_info.shape)
sample_info.head()

(71, 3070)


,!Sample_title,F1,F2,F3,F4,F5,F6,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40,F41,...,F2069repl,F2151repl,F2174repl,F2175repl,F2277repl,F2281repl,F2287repl,F2299repl,F2359repl,F2390repl,F2398repl,F2406repl,F2407repl,F2408repl,F2424repl,F2459repl,F2482repl,F2503repl,F2539repl,F2621repl,F2625repl,F2632repl,F2656repl,F2679repl,F2691repl,F2703repl,F2755repl,F2781repl,F2790repl,F2840repl,F2841repl,F2860repl,F2872repl,F2877repl,F2911repl,F2912repl,F2958repl,F3057repl,F3085repl,F3135repl
0,!Sample_geo_accession,GSM2528079,GSM2528080,GSM2528081,GSM2528082,GSM2528083,GSM2528084,GSM2528087,GSM2528088,GSM2528089,GSM2528090,GSM2528091,GSM2528092,GSM2528093,GSM2528094,GSM2528095,GSM2528096,GSM2528097,GSM2528098,GSM2528099,GSM2528100,GSM2528101,GSM2528102,GSM2528103,GSM2528104,GSM2528105,GSM2528106,GSM2528107,GSM2528108,GSM2528109,GSM2528110,GSM2528111,GSM2528112,GSM2528113,GSM2528114,GSM2528115,GSM2528116,GSM2528117,GSM2528118,GSM2528119,...,GSM2531438,GSM2531439,GSM2531440,GSM2531441,GSM2531442,GSM2531443,GSM2531444,GSM2531445,GSM2531446,GSM2531448,GSM2531449,GSM2531450,GSM2531451,GSM2531452,GSM2531453,GSM2531454,GSM2531455,GSM2531456,GSM2531457,GSM2531458,GSM2531459,GSM2531460,GSM2531461,GSM2531462,GSM2531463,GSM2531464,GSM2531465,GSM2531466,GSM2531467,GSM2531468,GSM2531469,GSM2531470,GSM2531472,GSM2531473,GSM2531474,GSM2531475,GSM2531477,GSM2531481,GSM2531483,GSM2531485
1,!Sample_status,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,...,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018,Public on Mar 12 2018
2,!Sample_submission_date,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,...,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Mar 09 2017,Ma

In [21]:
#removing replicates
samples=list(sample_info.columns)
sample_names=[itm for itm in samples if 'repl' not in itm]
print(len(sample_names))

2970


In [22]:
# Fitering samples with known meta data
final_samples=intersect(sample_names,count_data.columns)
print(len(final_samples))

2969


In [23]:
#data label distribution
sample_info.loc[18,:].value_counts()

nhg: G2                        1439
nhg: G3                        1115
nhg: G1                         454
nhg: NA                          61
!Sample_characteristics_ch1       1
Name: 18, dtype: int64

In [24]:
label=sample_info.loc[18,final_samples]

In [25]:

final_samples.insert(0,'Unnamed: 0')

In [26]:
#obtaining final subset
final_data=count_data[final_samples]

In [27]:


final_data.head()

,Unnamed: 0,F581,F1588,F2514,F3024,F742,F433,F2972,F2252,F1889,F1841,F2487,F1899,F851,F580,F1727,F3077,F1851,F48,F3206,F2037,F1082,F732,F1209,F1904,F1208,F1875,F2139,F834,F1482,F652,F1870,F2793,F1318,F2837,F1947,F285,F10,F867,F710,...,F1419,F94,F11,F802,F2464,F3226,F2855,F412,F1435,F635,F3236,F982,F3255,F786,F622,F3262,F2314,F2348,F3143,F686,F3199,F1280,F1350,F1897,F1166,F1178,F2854,F770,F2010,F1252,F1067,F3134,F1467,F237,F3252,F459,F1481,F2089,F2504,F2614
0,5_8S_rRNA,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,1.150443,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
1,5S_rRNA,2.753799,4.151136,2.966388,-3.321928,1.117269,-3.321928,-3.321928,-3.321928,3.317194,-3.321928,3.761179,-3.321928,2.313115,-3.321928,3.091812,-3.321928,7.019121,4.705625,-3.321928,-3.321928,3.854694,-3.321928,3.232690,4.006807,-3.321928,-3.321928,3.234734,3.636288,-3.321928,-3.321928,-3.321928,6.619650,5.250981,2.277226,-3.321928,3.699707,3.223401,-3.321928,2.799989,...,3.866127,2.572009,-3.321928,-3.321928,-3.321928,3.448082,0.994848,-3.321928,3.874885,-3.321928,-3.321928,5.527947,4.536414,-3.321928,-3.321928,-3.321928,3.847025,-3.321928,3.510546,-3.321928,-3.321928,-3.321928,-3.321928,4.104194,-3.321928,-3.321928,2.711499,5.918193,-3.321928,-3.321928,-3.321928,-3.321928,3.507008,3.329378,-3.321928,-3.321928,5.842639,-3.321928,4.384927,3.688270
2,6M1-18,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
3,7M1-2,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
4,7SK,0.261020,-0.440483,1.106167,-0.855126,-0.002223,-0.496623,-0.318547,-0.381543,0.218397,0.429626,1.457255,-0.633423,-0.333994,-0.363975,-0.087082,-0.817074,1.507530,0.437835,-0.003597,-0.376982,1.626568,-0.961735,2.209811,0.978863,1.918848,0.179264,0.392453,1.048078,1.185351,-0.430871,0.333649,1.522711,-0.268989,-1.348589,-1.487593,-0.965666,0.749841,-0.27

In [28]:
labels=label[final_data.columns[1:]]

In [29]:

final_data.loc['Labels']=labels

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [30]:
final_data.tail()

,Unnamed: 0,F581,F1588,F2514,F3024,F742,F433,F2972,F2252,F1889,F1841,F2487,F1899,F851,F580,F1727,F3077,F1851,F48,F3206,F2037,F1082,F732,F1209,F1904,F1208,F1875,F2139,F834,F1482,F652,F1870,F2793,F1318,F2837,F1947,F285,F10,F867,F710,...,F1419,F94,F11,F802,F2464,F3226,F2855,F412,F1435,F635,F3236,F982,F3255,F786,F622,F3262,F2314,F2348,F3143,F686,F3199,F1280,F1350,F1897,F1166,F1178,F2854,F770,F2010,F1252,F1067,F3134,F1467,F237,F3252,F459,F1481,F2089,F2504,F2614
30861,ZYG11B,2.60156,1.81255,2.68442,2.44852,2.37997,2.1995,2.32913,1.86916,2.79472,2.32415,2.09382,2.4527,2.65494,2.1962,2.393,2.58349,1.89238,2.76217,2.76313,2.54724,2.42177,1.87848,2.06295,2.76863,2.37526,2.72337,1.81522,2.78424,1.74727,2.34174,2.10513,2.35857,1.797,2.58726,2.95763,2.44163,1.81619,2.40362,2.29607,...,2.18829,2.38671,2.56857,2.2764,2.79538,2.15254,1.57794,1.7877,2.64017,2.51825,2.40884,2.68401,2.95703,2.36766,2.56102,2.30521,2.57096,2.35565,2.47606,2.37106,2.37883,2.76565,2.45672,2.50682,2.79911,2.41111,1.78966,2.69838,1.99333,2.26778,2.91536,1.97323,1.86559,2.69008,2.59494,2.31436,2.41407,1.49154,1.70286,2.17536
30862,ZYX,7.0365,6.43445,5.02512,6.79154,6.36697,6.26773,5.90667,5.4915,6.05207,6.26029,6.20087,6.10213,5.37734,6.90214,6.55254,4.66073,6.54344,6.82027,5.00293,5.74721,6.34266,7.08691,6.85905,6.72539,6.75151,6.78141,6.61298,5.50378,5.87589,6.57036,6.59259,6.65773,5.93391,6.71806,6.27375,4.7805,6.77248,6.47248,6.05287,...,7.46181,5.5454,6.2224,5.74909,6.92676,5.78863,5.00715,6.21539,7.1294,6.74534,6.15123,5.76393,6.12003,6.76069,6.94297,6.60472,6.55735,6.61357,6.52455,6.67258,6.81374,5.62079,5.95025,6.65482,6.92751,6.36671,6.14931,5.84323,6.82973,5.9489,5.37458,5.86478,6.97061,5.51628,6.56512,7.06077,6.59024,8.08732,6.08596,5.95232
30863,ZZEF1,2.8124,2.42275,2.9685,2.95229,2.50002,3.40341,2.65939,2.70795,2.81148,2.19401,1.49503,2.84104,1.52594,1.73229,2.16335,2.00704,2.42441,1.59576,1.81445,2.53874,2.14554,1.54646,2.27298,2.75838,1.79287,1.56329,2.09006,2.13656,0.768258,2.43191,2.12929,2.74545,1.76545,2.55876,1.70199,2.72596,2.19576,2.57988,2.10553,...,1.79943,2.45344,2.63452,2.79643,2.85461,2.36933,2.12793,2.34767,2.19938,2.54464,2.71761,1.73996,2.95587,1.74736,2.69439,2.04388,2.35122,2.56107,2.42087,2.71434,2.4186,2.64013,2.43089,2.43565,2.65018,2.4824,1.78112,1.72695,1.90627,2.4316,2.97964,2.70666,1.59204,2.23455,2.54137,1.73466,2.11741,2.17547,2.60305,2.33692
30864,ZZZ3,3.02973,2.91689,3.30785,2.97332,3.88406,2.94784,3.45923,3.71144,4.48847,3.33428,3.45633,3.51036,3.32961,2.7025,3.12836,3.61085,2.43043,4.13614,3.81267,3.48014,4.00644,3.01034,2.52929,3.36296,3.73937,3.63914,2.63725,3.87741,3.01809,3.20088,2.3703,3.26475,3.68274,2.61809,3.51341,3.69899,2.2017,3.87792,3.18262,...,2.76032,3.41169,3.38436,4.22637,3.87654,2.89278,3.63985,2.98951,3.59409,3.59103,2.70991,2.72393,4.09314,3.99172,3.66582,3.60891,2.9503,3.176,2.81023,3.28265,3.40778,4.05187,3.47742,4.01437,3.58148,3.40637,3.72501,4.34843,3.00705,4.14379,4.29088,3.5551,2.35651,3.71247,3.3462,3.49264,2.57913,2.5177,2.75516,2.51171
Labels,NaN,nhg: G2,nhg: G2,nhg: G1,nhg: G2,nhg: G2,nhg: NA,nhg: G2,nhg: G2,nhg: G1,nhg: G2,nhg: G3,nhg: G2,nhg: G3,nhg: G2,nhg: G3,nhg: G3,nhg: G2,nhg: G3,nhg: G2,nhg: G2,nhg: G2,nhg: G3,nhg: G2,nhg: G3,nhg: G2,nhg: G2,nhg: G2,nhg: G1,nhg: G3,nhg: G3,nhg: G2,nhg: G1,nhg: G3,nhg: G2,nhg: G2,nhg: G3,nhg: G3,nhg: G2,nhg: G2,...,nhg: G2,nhg: G3,nhg: G2,nhg: G2,nhg: G1,nhg: G2,nhg: G2,nhg: G3,nhg: G3,nhg: G1,nhg: G2,nhg: G3,nhg: G2,nhg: G3,nhg: G1,nhg: G2,nhg: G3,nhg: G3,nhg: G3,nhg: G2,nhg: G2,nhg: G2,nhg: G1,nhg: G1,nhg: G2,nhg: G1,nhg: G2,nhg: G3,nhg: G3,nhg: G2,nhg: G2,nhg: G3,nhg: G2,nhg: G2,nhg: G1,nhg: G1,nhg: G2,nhg: G3,nhg: G2,nhg: G3


In [31]:
final_data.at['Labels','Unnamed: 0']='Classes'

In [34]:
#creating an index list of missing label information
indx=~(final_data.loc['Labels']=='nhg: NA')

In [35]:
sum(indx)

2911

In [36]:
raw_data=final_data.loc[:,indx]

In [37]:
raw_data.rename(columns={'Unnamed: 0':'Sample_id'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
raw_data=raw_data.set_index('Sample_id')

In [39]:
raw_data=raw_data.T

In [43]:
raw_data.head()

Sample_id,5_8S_rRNA,5S_rRNA,6M1-18,7M1-2,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,A-589H1.1,AA06,AAAS,AACS,AACSP1,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,...,ZSCAN12P1,ZSCAN16,ZSCAN16-AS1,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN26,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,Classes
F581,-3.32193,2.7538,-3.32193,-3.32193,0.26102,2.40644,0.196553,-3.19712,7.78542,-1.44568,-2.66562,-2.76076,-3.32193,3.89666,-3.32193,-1.15058,-3.32193,4.17917,3.62901,-3.32193,-2.3697,-3.32193,-3.32193,-3.32193,-0.130202,3.04541,4.31235,2.66021,5.25336,5.75771,-2.54914,4.40788,3.53309,4.86539,2.01253,3.53668,1.93002,3.41077,0.262931,4.36893,...,-1.51607,0.404146,1.04374,4.53123,1.43172,-0.273801,1.56605,0.0794771,-1.28533,1.66381,1.87712,1.42855,1.64927,1.01522,2.05593,-2.98879,0.430571,-1.85707,1.03289,2.47605,-3.32193,0.736889,2.61523,-0.0198767,2.2459,2.56007,3.98193,1.65728,2.26815,2.13552,3.06896,0.612041,1.98643,2.87222,-2.29701,2.60156,7.0365,2.8124,3.02973,nhg: G2
F1588,-3.32193,4.15114,-3.32193,-3.32193,-0.440483,2.68925,-0.214284,-3.23383,6.71089,-1.70786,-1.1741,-3.32193,-3.32193,2.81136,-2.37423,-0.698712,-3.32193,3.69744,5.02585,-3.32193,-0.679529,-3.32193,-3.18579,-3.32193,2.01339,2.52502,4.67873,2.63752,9.34919,5.89478,-2.31957,4.20107,3.01935,4.96348,1.80558,3.93864,2.17123,3.21778,0.128678,4.51753,...,-1.05457,2.28203,2.07912,3.57872,1.8063,-0.418731,1.80401,0.180402,-1.65365,1.5195,2.27413,1.94437,1.87975,1.3714,2.04965,-3.08719,0.066931,-2.93124,1.60606,1.87527,-3.32193,0.404356,2.11246,-0.545018,1.00953,2.44793,3.76945,1.63078,2.3751,1.83595,3.19231,0.773245,1.83925,3.31224,-0.802543,1.81255,6.43445,2.42275,2.91689,nhg: G2
F2514,-3.32193,2.96639,-3.32193,-3.32193,1.10617,2.60117,0.285808,-3.21868,6.36642,-0.853298,-2.34706,-2.52524,-3.32193,2.75311,-3.32193,-0.658426,-3.32193,3.79055,3.73123,-3.32193,-2.84645,-3.32193,-3.32193,-3.32193,0.924709,2.60221,4.13105,2.75716,5.04116,5.7401,-3.04678,3.89795,0.700289,5.69016,1.22035,3.86619,2.06335,2.86123,0.723947,4.16202,...,-0.0658715,1.89087,1.89339,4.33391,2.35883,-0.536687,1.74654,0.111505,-2.72259,0.655481,2.55374,2.0456,1.37485,2.27341,2.36738,-3.05394,0.605701,-3.32193,1.64823,2.07435,-3.32193,0.58023,1.23171,1.92106,4.59073,3.51777,3.99773,2.05125,2.6752,0.690584,1.29065,0.832688,1.84868,3.34554,-2.96007,2.68442,5.02512,2.9685,3.30785,nhg: G1
F3024,-3.32193,-3.32193,-3.32193,-3.32193,-0.855126,2.01108,0.169288,-3.22635,7.50487,-1.92947,-2.61881,-3.32193,-3.32193,4.15618,-3.32193,-0.325548,-3.32193,4.40004,4.13241,-3.32193,-3.32193,-3.32193,-3.32193,-3.32193,-0.135586,2.34722,4.40847,3.11758,4.04958,5.83055,-3.32193,4.11178,1.4855,4.93233,2.01031,3.79412,1.97389,3.80158,-0.290334,4.65428,...,-1.44056,1.6479,1.10215,4.27751,1.66259,0.232499,1.52316,0.383918,-2.14363,1.90128,2.22403,1.83363,1.72781,1.12617,3.08642,-3.32193,0.516527,-3.32193,1.78271,2.08031,-3.32193,0.73663,3.06315,0.43726,2.33716,2.78201,4.07898,1.48801,2.33732,2.02328,3.65896,0.332633,1.94685,3.15399,-1.6134,2.44852,6.79154,2.95229,2.97332,nhg: G2
F742,-3.32193,1.11727,-3.32193,-3.32193,-0.00222342,2.71235,0.451235,-3.32193,5.97998,-1.81108,-3.08246,-3.32193,-3.32193,2.28903,-2.74937,-0.854589,-3.32193,3.63933,3.62291,-3.32193,-0.887409,-3.32193,-3.32193,-3.32193,-0.678333,2.81696,4.67915,2.22491,8.0321,6.31123,-2.93918,3.59308,1.45829,4.53276,1.79719,3.52108,2.06627,3.02263,-0.117541,4.25101,...,-0.268269,1.76488,2.0602,2.73401,1.76842,0.264792,2.23147,-0.13171,-0.257059,1.91596,2.00381,1.85879,1.9528,1.33007,2.40769,-3.32193,-0.0229075,-3.32193,1.51704,1.38174,-3.32193,-0.439246,1.72747,-1.74936,1.29345,2.51391,3.83245,1.43636,2.42726,2.14832,3.43784,0.481526,1.59148,2.89255,0.78911,2.37997,6.36697,2.50002,3.88406,nhg: G2


In [42]:
#saving the final file in current directory
raw_data.to_csv('raw_data_hist.csv',index=True)

# Step 2: Creating training data for autoencoder

In [ ]:
st=time.time()
data=pd.read_csv('raw_data_hist.csv',index_col=0)
end=time.time()
print(end-st)

In [ ]:
train, test = train_test_split(data, test_size=0.1, random_state=0, 
                               stratify=data['Classes'])

In [ ]:
(train.shape,test.shape)

(2619, 30866)
(291, 30866)

In [ ]:
train.to_csv('Training.csv')
test.to_csv('Test.csv')

# Step 3:Training autoencoder with few variations

In [ ]:
#creating a baseline model
%% bash
python -m Training_simple_model

In [ ]:
#creating a dense model
%% bash
python -m Training_dense_model

# Step 4: Testing saved models


In [ ]:
%% bash
python -m testing_saved_models